# Gather CPU Stats

This is a quick way of grabbing the last 256 minutes of CPU usage data from SQL Server and plotting it using R.

The SQL Server script comes from [Glenn Berry’s outstanding set of DMV queries](https://www.sqlskills.com/blogs/glenn/category/dmv-queries/).  This gives us back a total of 256 records, one for each of the past 256 minutes.

We will load RODBC to make an ODBC connection to SQL Server.  We will use tidyverse to clean up the resulting data, and will use ggplot2 to plot the data.

In [ ]:
if(!require(RODBC)) {
    install.packages("RODBC")
    library(RODBC)
}
 
if(!require(ggplot2)) {
    install.packages("ggplot2")
    library(ggplot2)
}
 
if(!require(tidyverse)) {
    install.packages("tidyverse")
    library(tidyverse)
}

Fill in your own connection string settings here.

In [ ]:
conn <- odbcDriverConnect("Driver=SQL Server;Server=.;Initial Catalog=master;Provider=SQLNCLI11.1;Integrated Security=SSPI")

Here's Glenn's query, which we've brought inline.

In [ ]:
cpu_query <- paste("DECLARE @ts_now BIGINT = (SELECT cpu_ticks / (cpu_ticks / ms_ticks) FROM sys.dm_os_sys_info WITH (NOLOCK) );
 
SELECT TOP (256)
    SQLProcessUtilization AS SQLCPU,
    SystemIdle AS Idle,
    100 - SystemIdle - SQLProcessUtilization AS OtherCPU,
    DATEADD(ms, -1 * (@ts_now - [TIMESTAMP]), GETDATE()) AS EventTime
FROM (
        SELECT
            record.value('(./Record/@id)[1]', 'int') AS record_id,
            record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int')
            AS [SystemIdle],
            record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int')
            AS [SQLProcessUtilization],
            [TIMESTAMP]
        FROM (
            SELECT
                [TIMESTAMP],
                CONVERT(XML, record) AS [record]
            FROM sys.dm_os_ring_buffers WITH (NOLOCK)
            WHERE
                ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
                AND record LIKE N'%<SystemHealth>%'
        ) AS X
    ) AS Y
ORDER BY
    record_id DESC OPTION (RECOMPILE);")

Now we run the query:

In [ ]:
cpu_usage_by_minute <- sqlQuery(conn, cpu_query)

`cpu_usage_by_minute` is a data frame which has variables named just like the result set of our query.  SQLCPU is the percent of CPU (0-100) used by SQL Server per minute.  OtherCPU is the percentage used by other processes.  Idle is the idle percentage.  And EventTime is the moment that the CPU data was added to the ring buffer.

In [ ]:
str(cpu_usage_by_minute)

Now we want to clean up the data a bit.  First, let's get the total CPU usage, which is simply SQLCPU + OtherCPU.  Then, I want to convert the date/time from POSIXct format to a date in the form YYYYmmDD_hhmmss (e.g., 20170621_185100).  That will make it easier to plot as a date/time.

Finally, we will unpivot the SQLCPU, OtherCPU, and TotalCPU values.  We're going to convert these from variables into a single variable called CPUType, and the value inside will be called PercentUse.

In [ ]:
cpu_usage_by_minute$TotalCPU <- cpu_usage_by_minute$SQLCPU + cpu_usage_by_minute$OtherCPU
cpu_usage_by_minute$EventTime <- lubridate::ymd_hms(cpu_usage_by_minute$EventTime)
cpu_usage_by_minute_plot_df <- cpu_usage_by_minute %>%
    gather(key = CPUType, value = PercentUse, SQLCPU, OtherCPU, TotalCPU) %>%
    select(EventTime, CPUType, PercentUse)

Now that we have cleaned up the data a bit, we can plot the results.

We will plot the data in `cpu_usage_by_minute_plot_df`.  With this data in mind, we want to create an *aesthetic*, laying out that the x axis will show EventTime and the y axis will represent PercentUse.  We want to break the lines out by CPU type and color them differently.

Finally, I want to include the range from 0% to 100% on the y axis.  The reason I want to specify this range is to prevent playing tricks on ourselves (or anyone else!).  If we allow ggplot to define the relevant range, it will find a reasonable-looking visual, but will have arbitrary begin and end points.  It's possible for me to take two graphics and make it look like there is a significant difference simply because the axes are different.  By fixing the range like this, we can ensure that no matter how many times we run the process and generate an image, we will be able to eyeball the differences between images and know that those differences are real and not simply chimerical.

In [ ]:
ggplot(cpu_usage_by_minute_plot_df, aes(x = EventTime, y = PercentUse, color = CPUType, ymin = 0, ymax = 100)) +
    geom_line(stat='identity')

Fun notes around this.  First, if you are running this from a laptop which gets put to sleep regularly, you can have jumps in the data.  Also, if you're running this from a lightly-loaded dev box, your results are probably going to be pretty boring.  Here's an example of a not-quite-as-boring server:

<img src="Files/cpuusage.png" width="660" height="420" alt="Proof that I hate my servers." />

As a quick note, this was an earlier version of the image, back before I added in total CPU and hard limits on ymin and ymax.